# Environment

So far we did not worry very much about where the data came from and how the models that we build get deployed. Not caring about it can be problematic. Many failed machine learning deployments can be traced back to this situation. This chapter is meant to help with detecting such situations early and points out how to mitigate them. Depending on the case this might be rather simple (ask for the 'right' data) or really difficult (implement a reinforcement learning system). 

## Covariate Shift

At its heart is a problem that is easy to understand but also equally easy to miss. Consider being given the challenge of distinguishing cats and dogs. Our training data consists of images of the following kind:

|![](img/cat1.jpg)|![](img/cat2.jpg)|![](img/dog1.jpg)|![](img/dog2.jpg)|
|:---------------:|:---------------:|:---------------:|:---------------:|
|cat|cat|dog|dog|

At test time we are asked to classify the following images:

|![](img/cat-cartoon1.png)|![](img/cat-cartoon2.png)|![](img/dog-cartoon1.png)|![](img/dog-cartoon2.jpg)|
|:---------------:|:---------------:|:---------------:|:---------------:|
|cat|cat|dog|dog|

Obviously this is unlikely to work well. The training set consisted of photos, while the test set contained cartoons only. Not even the colors are accurate. Who would be so stupid to do something like that! Training on a dataset that looks substantially different from the test set without any further safeguards is a bad idea. Unfortunately, this is a very common pitfall. Statisticians call this **Covariate Shift**, i.e. the situation where the distribution over the covariates (aka training data) is shifted on test data relative to the training case. Mathematically speaking, we are referring the case where $p(x)$ changes but $p(y|x)$ remains unchanged.

## Concept Shift

A related problem is that of concept shift. This is the situation where the the labels change. This sounds weird - after all, a cat is a cat is a cat. Well, cats maybe but not soft drinks. There is considerable concept shift throughout the USA, even for such a simple term:

![](img/sodapopcoke.png)

If we were to build a machine translation system, the distribution $p(y|x)$ would be different, e.g. depending on our location. This problem can be quite tricky to spot. A saving grace is that quite often the $p(y|x)$ only shifts gradually (e.g. the click-through rate for NOKIA phone ads). Before we go into further details, let us discuss a number of situations where covariate and concept shift are not quite as blatantly obvious.


## Examples

### Medical Diagnostics

Imagine you want to design some algorithm to detect cancer. You get data of healthy and sick people; you train your algorithm; it works fine, giving you high accuracy and you conclude that you’re ready for a successful career in medical diagnostics. Not so fast ...

Many things could go wrong. In particular, the distributions that you work with for training and those in the wild might differ considerably. This happened to an unfortunate startup I had the opportunity to consult for many years ago. They were developing a blood test for a disease that affects mainly older men and they’d managed to obtain a fair amount of blood samples from patients. It is considerably more difficult, though, to obtain blood samples from healthy men (mainly for ethical reasons). To compensate for that, they asked a large number of students on campus to donate blood and they performed their test. Then they asked me whether I could help them build a classifier to detect the disease. I told them that it would be very easy to distinguish between both datasets with probably near perfect accuracy. After all, the test subjects differed in age, hormone level, physical activity, diet, alcohol consumption, and many more factors unrelated to the disease. This was unlikely to be the case with real patients: Their sampling procedure had caused an extreme case of covariate shift that couldn’t be corrected by conventional means. In other words, training and test data were so different that nothing useful could be done and they had wasted significant amounts of money. 

### Self Driving Cars

A company wanted to build a machine learning system for self-driving cars. One of the key components is a roadside detector. Since real annotated data is expensive to get, they had the (smart and questionable) idea to use synthetic data from a game rendering engine as additional training data. This worked really well on 'test data' drawn from the rendering engine. Alas, inside a real car it was a disaster. As it turned out, the roadside had been rendered with a very simplistic texture. More importantly, *all* the roadside had been rendered with the *same* texture and the roadside detector learned about this 'feature' very quickly. 

A similar thing happened to the US Army when they first tried to detect tanks in the forest. They took aerial photographs of the forest without tanks, then drove the tanks into the forest and took another set of pictures. The so-trained classifier worked 'perfectly'. Unfortunately, all it had learned was to distinguish trees with shadows from trees without shadows - the first set of pictures was taken in the early morning, the second one at noon. 

### Nonstationary distributions

A much more subtle situation is where the distribution changes slowly and the model is not updated adequately. 

In [ ]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
ctx = mx.gpu()


In general the situation is not quite so dire. Assume that we want to estimate some dependency p(y|x)p(y|x) for which we have labeled data (xi,yi)(xi,yi). Alas, the observations xixi are drawn from some distribution q(x)q(x) rather than the ‘proper’ distribution p(x)p(x). If we adopt a risk minimization approach, that is, if we want to solve
minimizef1m∑i=1ml(xi,yi,f(xi))+λ2‖f‖2
minimizef1m∑i=1ml(xi,yi,f(xi))+λ2‖f‖2
we will need to re-weight each instance by the ratio of probabilities that it would have been drawn from the correct distribution, that is, we need to reweight things by p(xi)q(xi)p(xi)q(xi). This is the ratio of how frequently the instances would have occurred in the correct set vs. how frequently it occurred with the sampling distribution qq. It is sometimes also referred to as the Radon-Nikodym derivative. Such a method is called importance sampling and the following derivation shows why it is valid:
∫f(x)dp(x)=∫f(x)dp(x)dq(x)dq(x)
∫f(x)dp(x)=∫f(x)dp(x)dq(x)dq(x)
Alas, we do not know dp(x)dq(x)dp(x)dq(x), so before we can do anything useful we need to estimate the ratio. Many methods are available, e.g. some rather fancy operator theoretic ones which try to recalibrate the expectation operator directly using a minimum-norm or a maximum entropy principle. However, there exists a much more pedestrian, yet quite effective approach that will give almost as good results: logistic regression. 
After all, we know how to estimate probability ratios. This is achieved by learning a classifier to distinguish between data drawn from pp and data drawn from qq. If it is impossible to distinguish between the two distributions then it means that the associated instances are equaly likely to come from either oneof the two distributions. On the other hand, any instances that can be well discriminated should be significantly over/underweighted accordingly. For simplicity’s sake assume that we have an equal number of instances from both distributions, denoted by xi∼p(x)xi∼p(x) and x′i∼q(x)xi′∼q(x) respectively. Now denote by zizi labels which are 1 for data drawn from pp and -1 for data drawn from qq. Then the probability in a mixed dataset is given by
p(z=1|x)=p(x)p(x)+q(x)
p(z=1|x)=p(x)p(x)+q(x)
Hence, if we use a logistic regression approach which yields p(z=1|x)=11+e−f(x)p(z=1|x)=11+e−f(x), it follows (after some simple algebra) that 
p(z=1|x)p(z=−1|x)=ef(x).
p(z=1|x)p(z=−1|x)=ef(x).
Now we only need to solve the logistic regression problem
minimizef12m∑(x,z)log[1+exp(−zf(x))]+λ2‖f‖2
minimizef12m∑(x,z)log⁡[1+exp⁡(−zf(x))]+λ2‖f‖2
to obtain ff. Subsequently we can use ef(xi)ef(xi) as covariate shift correction weights in training our actual classifier. The good news is that we can use an off-the-shelf tool such as logistic regression to deal with a decidedly nonstandard estimation problem. 

## The MNIST Dataset

In [ ]:
mnist = mx.test_utils.get_mnist()
num_examples = 1000
batch_size = 64
train_data = mx.io.NDArrayIter(
    mnist["train_data"][:num_examples], 
    mnist["train_label"][:num_examples], 
    batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(
    mnist["test_data"][:num_examples], 
    mnist["test_label"][:num_examples], 
    batch_size, shuffle=True)

## Multiclass Logistic Regression

In [ ]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(10))